#### PreProcessing -Fundamentals by Industry
Scaled fundamental data is concatenated by industry to allow training on all stocks in a given industry. This may help ovrefitting issues.

In [ ]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy as np
import os
import urllib
import json
import requests
import matplotlib.pyplot as plt
from io import StringIO

from pathlib import Path
from eod import EodHistoricalData

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)

In [ ]:
#paths for data - set prefix to location of Data folder
path_prefix = r'C:\Users\OEM\GDrive\WQU'
path_att = r'\Data\fundamentals_by_attribute'
path_fun = r'\Data\fundamentals_by_ticker'
path_std = r'\Data\standardised_fundamentals'
path_eda = r'\Data\exploratory_data_analysis'
path_rob = r'\Data\robust_scaling'
path_ind = r'\Data\indicator_prices'
path_sec = r'\Data\rob_scaled_data_by_sector'

In [ ]:
#Instantiate datasource client
api_key = "618f834b7bfe27.18132752"
client = EodHistoricalData(api_key)

In [ ]:
#get ticker symbols from exchange
resp = client.get_exchange_symbols(exchange='NYSE')

In [ ]:
#create list of stock tickers
tickers = []
for i in range(len(resp)):
    if resp[i]['Type'] == 'Common Stock' or resp[i]['Type'] == 'Preferred Stock' :
        ticker = resp[i]['Code']
        tickers.append(ticker)

In [ ]:
industries = {}
for ticker in tickers:
    industry = client.get_fundamental_equity(ticker)['General']['Industry']
    industries[ticker] = industry
    df_industries = pd.Series(industries)
    df_industries.to_csv(path_prefix + "\industries_by_ticker")
    print(ticker)

In [ ]:
df_industries = pd.read_csv(path_prefix + "\industries_by_ticker", index_col=0, names=['Industry'])

In [ ]:
for ticker in tickers:
    filepath = Path(path_prefix + path_rob + '/df_rob_{}.csv'.format(ticker))
    
    if os.path.isfile(filepath) == True:
        df_rob = pd.read_csv(filepath, index_col=0)
        industry = df_industries.loc[ticker, 'Industry']
        
        filepath = Path(path_prefix + path_sec + '/df_{}.csv'.format(industry))
        
        if os.path.isfile(filepath) == False:
            
            filepath.parent.mkdir(parents=True, exist_ok=True)  
            df_rob.to_csv(filepath)
            
        else:  
            
            df_rob_sec = pd.read_csv(filepath, index_col=0)
            concat = pd.concat([df_rob_sec, df_rob])
            concat.to_csv(filepath)
            
            print(ticker + " complete")

In [ ]:
resp = client.get_fundamental_equity('AAP')['General']['Industry']

In [ ]:
resp

In [ ]:
os.chdir(path_prefix + path_ind)

In [ ]:
files = [f for f in os.listdir() if os.path.isfile(f)]

In [ ]:
for file in files:
    filepath = path_prefix + path_rob + "/{}".format(file)
    try:
        if os.path.isfile(filepath) == True and file != 'desktop.ini':
            df_rob = pd.read_csv(filepath, index_col=0)
            df_rob.index = pd.to_datetime(df_rob.index)
            df_rob = df_rob.loc[df_rob.index > earliest_date]
            df_rob.to_csv(filepath)
    except:
        print("Error on " + filepath)
        
    
    print(filepath + " amended")